<a href="https://colab.research.google.com/github/Joseasd213/Gatos-y-Perros/blob/main/Perro_vs_gato_xd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import os
import pathlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Descargar y preparar el dataset
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_path = tf.keras.utils.get_file('cats_and_dogs_filtered.zip', origin=url)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.dirname(zip_path))

base_dir = os.path.join(pathlib.Path(zip_path).parent, 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Preprocesamiento de datos
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    batch_size=10,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(100, 100),
    batch_size=10,
    class_mode='binary'
)

# Definir el modelo CNN
model = models.Sequential([
    layers.Conv2D(8, (3,3), activation='relu', input_shape=(100, 100, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(16, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compilación del modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenamiento del modelo
history = model.fit(
    train_generator,
    epochs=5,  # Puedes cambiarlo si quieres entrenar más
    validation_data=validation_generator
)

# Gráfica de precisión
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Entrenamiento')
plt.plot(epochs, val_acc, 'b', label='Validación')
plt.title('Precisión del modelo')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.grid(True)
plt.show()

# Guardar el modelo
model_json = model.to_json()
with open("model_gats_gossos.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_gats_gossos.weights.h5")

# Descargar los archivos del modelo
from google.colab import files
files.download("model_gats_gossos.json")
files.download("model_gats_gossos.weights.h5")

# Subir una imagen para predecir
from google.colab import files
uploaded = files.upload()

# Predecir con la imagen subida
from PIL import Image
image_name = list(uploaded.keys())[0]  # Obtiene el nombre de la imagen subida

# Preparar la imagen
image = Image.open(image_name).convert("RGB").resize((100, 100))
image_array = np.array(image) / 255.0
image_array = np.expand_dims(image_array, axis=0)  # Añadir dimensión batch

# Hacer la predicción
prediction = model.predict(image_array)
prob = float(prediction[0])

# Mostrar el resultado de la predicción
plt.imshow(image)
plt.axis('off')
if prob > 0.5:
    pred_label = f"🐶 ¡Es un *perro*! ({prob*100:.2f}%)"
    pred_clase = "perro"
else:
    pred_label = f"🐱 ¡Es un *gato*! ({(1 - prob)*100:.2f}%)"
    pred_clase = "gato"
plt.title(pred_label)
plt.show()

# Pedir retroalimentación al usuario
print(f"La IA predijo que es un {pred_clase}.")
feedback = input("¿Es correcta la predicción? (sí / no / dime cuál es): ").strip().lower()

if feedback == "si":
    print("✅ ¡Genial! Me alegra haber acertado. 😺🐶")
elif feedback == "no":
    print("❌ Gracias por avisarme. ¿Qué es realmente? (escribe 'gato' o 'perro')")
    correcion = input("Tu respuesta: ").strip().lower()
    if correcion == "gato" or correcion == "perro":
        print(f"✅ Gracias por corregirme. He anotado que es un {correcion}.")
    else:
        print("⚠️ No entendí tu respuesta, pero gracias por intentarlo.")
else:
    print("🤔 No entendí tu respuesta, pero gracias por participar.")




Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 19s 90ms/step - accuracy: 0.5027 - loss: 0.6972 - val_accuracy: 0.6390 - val_loss: 0.6674
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 15s 77ms/step - accuracy: 0.6463 - loss: 0.6356 - val_accuracy: 0.6500 - val_loss: 0.6239
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 15s 77ms/step - accuracy: 0.7379 - loss: 0.5509 - val_accuracy: 0.5960 - val_loss: 0.7248
Epoch 4/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 15s 77ms/step - accuracy: 0.7795 - loss: 0.4649 - val_accuracy: 0.6290 - val_loss: 0.7590
Epoch 5/5
173/200 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.8260 - loss: 0.3770